In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import matplotlib.pyplot as plt

In [2]:
mercado_df = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Mercado/data_consolidado_final_jun24.csv",
                     encoding='latin-1',
                     delimiter=';')

In [3]:
mercado_df.head(3)

,ano_mod_conv,classe_bonus,fabrica,Grupo_veiculo_new,i_0km,idade,idade_ult_RNS,md_categoria,MD_comb,md_config,...,LIBERTY,PORTO,TOKIO,ZURICH,premio_total_liq_cap,is_casco,is_rcdm,is_rcdc,is_dmoral,qtd
0,2013,10,FIAT,Fiat Palio 1.4,N,71,Sem Sinistro,10 - Passeio Nacional,Flex,03 - Hatchback,...,1328.921587,NaN,NaN,1188.959303,1061.563373,34526.0,100000.0,100000.0,5000.0,24
1,2009,10,CHEVROLET,Chevrolet Corsa Sedan 1.0,N,48,Sem Sinistro,10 - Passeio Nacional,Gasolina,10 - Sedan,...,1163.202645,NaN,NaN,944.471969,959.694356,18970.0,50000.0,50000.0,NaN,9
2,2014,10,HYUNDAI,Hyundai HB20 1.0,N,42,Sem Sinistro,10 - Passeio Nacional,Flex,03 - Hatchback,...,1114.229838,919.26057,NaN,1116.635314,993.330928,40420.0,100000.0,100000.0,NaN,26


In [91]:
df_mod=mercado_df[["premio_total_liq_cap","ano_mod_conv","classe_bonus",
                   "idade","is_casco","is_dmoral","is_rcdc","is_rcdm",
                   "RNS_Casco_calc",
                   "Grupo_veiculo_new", "fabrica","idade_ult_RNS","md_categoria",
                   "MD_comb","md_config","MD_franquia","md_menor26","md_sexo",
                   "MD_Subzona","md_tipo_renov","MD_uso_veic","uf"]]

In [95]:
display(df_mod.describe().T)
display(df_mod.describe(include=['O']).T)

,count,mean,std,min,25%,50%,75%,max
premio_total_liq_cap,96586.0,2642.426137,1472.963374,845.956416,1697.074409,2236.016414,3088.75433,18183.331471
ano_mod_conv,96586.0,2017.935953,3.960001,2004.000000,2015.000000,2018.000000,2021.00000,2025.000000
classe_bonus,96586.0,6.545949,3.406705,0.000000,3.000000,7.000000,10.00000,10.000000
idade,96586.0,48.775847,13.895189,18.000000,38.000000,47.000000,59.00000,97.000000
is_casco,96495.0,80977.882195,48882.449279,13887.000000,46565.000000,69679.000000,101995.00000,620800.000000
is_dmoral,70718.0,20826.817684,21017.365103,5000.000000,10000.000000,10000.000000,30000.00000,130000.000000
is_rcdc,96581.0,166578.847831,109039.010834,40000.000000,100000.000000,150000.000000,200000.00000,1000000.000000
is_rcdm,96582.0,155868.734581,85671.592733,30000.000000,100000.000000,150000.000000,200000.00000,1000000.000000
RNS_Casco_calc,96586.0,0.089371,0.317307,0.000000,0.000000,0.000000,0.00000,3.000000


,count,unique,top,freq
Grupo_veiculo_new,96586,197,Toyota Corolla,4630
fabrica,96586,28,CHEVROLET,16677
idade_ult_RNS,96586,4,Sem Sinistro,70681
md_categoria,96586,9,10 - Passeio Nacional,59013
MD_comb,96586,6,Flex,84440
md_config,96586,15,03 - Hatchback,30882
MD_franquia,96586,3,12 - Reduzida,91877
md_menor26,96586,2,NAO,88753
md_sexo,96586,2,Masculino,50100
MD_Subzona,96586,228,82100-METROPOLITANA DE GOIANIA,2657


In [37]:
print(df_mod['MD_uso_veic'].value_counts(ascending=False))

Particular                       94046
Visitar Clientes/Fornecedores     2504
Traslado Remunerado                 23
Entrega de Mercadoria               13
Name: MD_uso_veic, dtype: int64


In [5]:
df_mod.head(3)

,premio_total_liq_cap,ano_mod_conv,classe_bonus,idade,is_casco,is_dmoral,is_rcdc,is_rcdm,RNS_Casco_calc,Grupo_veiculo_new,...,md_categoria,MD_comb,md_config,MD_franquia,md_menor26,md_sexo,MD_Subzona,md_tipo_renov,MD_uso_veic,uf
0,1061.563373,2013,10,71,34526.0,5000.0,100000.0,100000.0,0,Fiat Palio 1.4,...,10 - Passeio Nacional,Flex,03 - Hatchback,12 - Reduzida,NAO,Masculino,88203-METROPOLITANA DE RECIFE - BOA VIAGEM,2 - Renov MAPFRE sem sinistro,Particular,CE
1,959.694356,2009,10,48,18970.0,NaN,50000.0,50000.0,0,Chevrolet Corsa Sedan 1.0,...,10 - Passeio Nacional,Gasolina,10 - Sedan,12 - Reduzida,NAO,Masculino,82100-METROPOLITANA DE GOIANIA,2 - Renov MAPFRE sem sinistro,Particular,GO
2,993.330928,2014,10,42,40420.0,NaN,100000.0,100000.0,0,Hyundai HB20 1.0,...,10 - Passeio Nacional,Flex,03 - Hatchback,12 - Reduzida,NAO,Feminino,42602-SOROCABA-LESTE,2 - Renov MAPFRE sem sinistro,Particular,SP


In [97]:
df_mod.rename(columns={'premio_total_liq_cap':'premio_ofertado', 'ano_mod_conv': 'ano_mod'},inplace=True)

C:\Users\flcoelho\AppData\Local\Temp\ipykernel_51124\2314202701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod.rename(columns={'premio_total_liq_cap':'premio_ofertado', 'ano_mod_conv': 'ano_mod'},inplace=True)


In [98]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96586 entries, 0 to 96585
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   premio_ofertado    96586 non-null  float64
 1   ano_mod            96586 non-null  int64  
 2   classe_bonus       96586 non-null  int64  
 3   idade              96586 non-null  int64  
 4   is_casco           96495 non-null  float64
 5   is_dmoral          70718 non-null  float64
 6   is_rcdc            96581 non-null  float64
 7   is_rcdm            96582 non-null  float64
 8   RNS_Casco_calc     96586 non-null  int64  
 9   Grupo_veiculo_new  96586 non-null  object 
 10  fabrica            96586 non-null  object 
 11  idade_ult_RNS      96586 non-null  object 
 12  md_categoria       96586 non-null  object 
 13  MD_comb            96586 non-null  object 
 14  md_config          96586 non-null  object 
 15  MD_franquia        96586 non-null  object 
 16  md_menor26         965

In [99]:
display(df_mod.describe().T)
display(df_mod.describe(include=['O']).T)

,count,mean,std,min,25%,50%,75%,max
premio_ofertado,96586.0,2642.426137,1472.963374,845.956416,1697.074409,2236.016414,3088.75433,18183.331471
ano_mod,96586.0,2017.935953,3.960001,2004.000000,2015.000000,2018.000000,2021.00000,2025.000000
classe_bonus,96586.0,6.545949,3.406705,0.000000,3.000000,7.000000,10.00000,10.000000
idade,96586.0,48.775847,13.895189,18.000000,38.000000,47.000000,59.00000,97.000000
is_casco,96495.0,80977.882195,48882.449279,13887.000000,46565.000000,69679.000000,101995.00000,620800.000000
is_dmoral,70718.0,20826.817684,21017.365103,5000.000000,10000.000000,10000.000000,30000.00000,130000.000000
is_rcdc,96581.0,166578.847831,109039.010834,40000.000000,100000.000000,150000.000000,200000.00000,1000000.000000
is_rcdm,96582.0,155868.734581,85671.592733,30000.000000,100000.000000,150000.000000,200000.00000,1000000.000000
RNS_Casco_calc,96586.0,0.089371,0.317307,0.000000,0.000000,0.000000,0.00000,3.000000


,count,unique,top,freq
Grupo_veiculo_new,96586,197,Toyota Corolla,4630
fabrica,96586,28,CHEVROLET,16677
idade_ult_RNS,96586,4,Sem Sinistro,70681
md_categoria,96586,9,10 - Passeio Nacional,59013
MD_comb,96586,6,Flex,84440
md_config,96586,15,03 - Hatchback,30882
MD_franquia,96586,3,12 - Reduzida,91877
md_menor26,96586,2,NAO,88753
md_sexo,96586,2,Masculino,50100
MD_Subzona,96586,228,82100-METROPOLITANA DE GOIANIA,2657


In [100]:
df_mod.columns.to_list()

['premio_ofertado',
 'ano_mod',
 'classe_bonus',
 'idade',
 'is_casco',
 'is_dmoral',
 'is_rcdc',
 'is_rcdm',
 'RNS_Casco_calc',
 'Grupo_veiculo_new',
 'fabrica',
 'idade_ult_RNS',
 'md_categoria',
 'MD_comb',
 'md_config',
 'MD_franquia',
 'md_menor26',
 'md_sexo',
 'MD_Subzona',
 'md_tipo_renov',
 'MD_uso_veic',
 'uf']

In [ ]:
#x_df, y = df_mod.iloc[:,1:],  df_mod.iloc[:,0]
#x_cat=pd.get_dummies(x_df.iloc[:,8:])
#x_df=pd.concat([x_df.iloc[:,0:7], x_cat],axis=1)
#x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15) 

In [103]:
x_df_num = df_mod[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
x_df_cat = df_mod[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
y   = df_mod[['premio_ofertado']]
x_df_dummy=pd.get_dummies(x_df_cat)
x_df=pd.concat([x_df_num, x_df_dummy],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15) 

['ano_mod',
 'classe_bonus',
 'idade',
 'is_casco',
 'is_dmoral',
 'is_rcdc',
 'is_rcdm',
 'RNS_Casco_calc',
 'Grupo_veiculo_new_ARRIZO',
 'Grupo_veiculo_new_Audi',
 'Grupo_veiculo_new_BMW Séries',
 'Grupo_veiculo_new_BMW X',
 'Grupo_veiculo_new_Chery Passeio',
 'Grupo_veiculo_new_Chery Tiggo',
 'Grupo_veiculo_new_Chevrolet Agile',
 'Grupo_veiculo_new_Chevrolet Astra Hatch',
 'Grupo_veiculo_new_Chevrolet Captiva',
 'Grupo_veiculo_new_Chevrolet Celta 1.0',
 'Grupo_veiculo_new_Chevrolet Celta 1.4',
 'Grupo_veiculo_new_Chevrolet Cobalt 1.4',
 'Grupo_veiculo_new_Chevrolet Cobalt 1.8',
 'Grupo_veiculo_new_Chevrolet Corsa Hatch 1.0',
 'Grupo_veiculo_new_Chevrolet Corsa Hatch 1.4+',
 'Grupo_veiculo_new_Chevrolet Corsa Sedan 1.0',
 'Grupo_veiculo_new_Chevrolet Corsa Sedan 1.4+',
 'Grupo_veiculo_new_Chevrolet Cruze Hatch',
 'Grupo_veiculo_new_Chevrolet Cruze Sedan',
 'Grupo_veiculo_new_Chevrolet Meriva',
 'Grupo_veiculo_new_Chevrolet Montana',
 'Grupo_veiculo_new_Chevrolet Onix 1.0',
 'Grupo_ve

In [104]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96586 entries, 0 to 96585
Columns: 535 entries, ano_mod to uf_TO
dtypes: float64(4), int64(4), uint8(527)
memory usage: 54.4 MB


In [105]:
x_df_cat.head(2)

,Grupo_veiculo_new,fabrica,idade_ult_RNS,md_categoria,MD_comb,md_config,MD_franquia,md_menor26,md_sexo,MD_Subzona,md_tipo_renov,MD_uso_veic,uf
0,Fiat Palio 1.4,FIAT,Sem Sinistro,10 - Passeio Nacional,Flex,03 - Hatchback,12 - Reduzida,NAO,Masculino,88203-METROPOLITANA DE RECIFE - BOA VIAGEM,2 - Renov MAPFRE sem sinistro,Particular,CE
1,Chevrolet Corsa Sedan 1.0,CHEVROLET,Sem Sinistro,10 - Passeio Nacional,Gasolina,10 - Sedan,12 - Reduzida,NAO,Masculino,82100-METROPOLITANA DE GOIANIA,2 - Renov MAPFRE sem sinistro,Particular,GO


In [106]:
x_df_cat.columns.to_list()

['Grupo_veiculo_new',
 'fabrica',
 'idade_ult_RNS',
 'md_categoria',
 'MD_comb',
 'md_config',
 'MD_franquia',
 'md_menor26',
 'md_sexo',
 'MD_Subzona',
 'md_tipo_renov',
 'MD_uso_veic',
 'uf']

In [107]:
x_df_num.columns.to_list()

['ano_mod',
 'classe_bonus',
 'idade',
 'is_casco',
 'is_dmoral',
 'is_rcdc',
 'is_rcdm',
 'RNS_Casco_calc']

In [108]:
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

In [109]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval  = lgb.Dataset(x_test, y_test, reference=lgb_train)

In [110]:
model_gbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)
y_pred = model_gbm.predict(x_test, predict_disable_shape_check=True)


In [150]:
y_pred = model_gbm.predict(x_test, predict_disable_shape_check=True)

# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 
 
MSE: 586716.47
RMSE: 765.97

MSE: 586716.47
RMSE: 765.97


In [160]:
df_describe = pd.DataFrame(y_pred)
df_describe.describe()

,0
count,14488.000000
mean,2656.054059
std,1218.433051
min,1054.606949
25%,1852.431519
50%,2340.250137
75%,3032.356652
max,10727.535257


## Adicionar GBM na base de modelagem GLM

In [172]:
df_mod_mp_feb23may24_v3 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23may24_v3.csv",
                     encoding='latin-1',
                     delimiter='|')

In [180]:
#df_mod_mp_feb23may24_v3.columns.to_list()

In [182]:
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[df_mod_mp_feb23may24_v3['per_com'].notna()]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[df_mod_mp_feb23may24_v3['Renovacao_ant'].notna()]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[df_mod_mp_feb23may24_v3['MD_est_civil'].notna()]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[df_mod_mp_feb23may24_v3['peso'].notna()]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[~df_mod_mp_feb23may24_v3['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3.query('teste_validacao == "Trein"')
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3.query('vigencia == "Anual"')
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3[(df_mod_mp_feb23may24_v3['per_com']>0) & (df_mod_mp_feb23may24_v3['per_com']<50)]
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3.query('premio_total >100')
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3.query('premio_creserva >= 0')
df_mod_mp_feb23may24_v3 = df_mod_mp_feb23may24_v3.query('is_casco_ofer > 0')
df_mod_mp_feb23may24_v3.shape

(310913, 105)

In [47]:
print(df_mod_mp_feb23may24_v3['MD_idade'].value_counts(ascending=False))

041    18188
042    17899
040    17350
043    17288
044    16612
       ...  
095       14
098        4
097        2
100        2
099        1
Name: MD_idade, Length: 83, dtype: int64


In [46]:
#df_mod_mp_feb23may24_v3.columns.to_list()

In [40]:
colunas=['premio_ofertado','ano_mod','classe_bonus','MD_idade','is_casco_ofer','is_dmoral','is_rcdc','is_rcdm','RNS_Casco_calc',
 'Grupo_veiculo_new','FABRICA','idade_ult_RNS', 'md_categoria','MD_comb','MD_config','MD_franquia','MD_menor26','MD_sexo','MD_Subzona',  
 'MD_tipo_renov','MD_uso_veic','UF']

In [183]:
df_mod_glm_mercado=df_mod_mp_feb23may24_v3[colunas]

In [184]:
df_mod_glm_mercado.rename(columns={'MD_idade':'idade', 
                                   'is_casco_ofer': 'is_casco',
                                   'FABRICA': 'fabrica',
                                   'MD_config':  'md_config',
                                   'MD_menor26': 'md_menor26',
                                   'MD_sexo': 'md_sexo',
                                   'UF': 'uf',
                                   'MD_tipo_renov':  'md_tipo_renov'},inplace=True)

In [185]:
df_mod_glm_mercado['idade'] = pd.to_numeric(df_mod_glm_mercado['idade'], errors='coerce')

In [186]:
df_mod_glm_mercado.columns.to_list()

['premio_ofertado',
 'ano_mod',
 'classe_bonus',
 'idade',
 'is_casco',
 'is_rcdm',
 'is_rcdc',
 'is_dmoral',
 'uf',
 'md_sexo',
 'RNS_Casco_calc',
 'Grupo_veiculo_new',
 'idade_ult_RNS',
 'fabrica',
 'md_categoria',
 'MD_comb',
 'md_config',
 'MD_Subzona',
 'md_tipo_renov',
 'MD_franquia',
 'md_menor26',
 'MD_uso_veic']

In [187]:
df_mod_glm_mercado.head(2)

,premio_ofertado,ano_mod,classe_bonus,idade,is_casco,is_rcdm,is_rcdc,is_dmoral,uf,md_sexo,...,idade_ult_RNS,fabrica,md_categoria,MD_comb,md_config,MD_Subzona,md_tipo_renov,MD_franquia,md_menor26,MD_uso_veic
355,1971.94,2005,10,70.0,24887.0,150000.0,400000.0,50000.0,PR,Masculino,...,01 ano,AUDI,10 - Passeio Nacional,Gasolina,05 - Hatchback Médio,32200-NORTE CENTRAL PARANAENSE-MARINGA,2 - Renov MAPFRE sem sinistro,12 - Reduzida,ERR,Particular
359,2283.56,2003,4,36.0,22863.0,100000.0,100000.0,20000.0,SP,Masculino,...,Sem Sinistro,AUDI,10 - Passeio Nacional,Gasolina,05 - Hatchback Médio,59404-RIBEIRAO PRETO-NORTE,2 - Renov MAPFRE sem sinistro,12 - Reduzida,NAO,Particular


In [188]:
df_mod_glm_mercado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310913 entries, 355 to 571593
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   premio_ofertado    310913 non-null  float64
 1   ano_mod            310913 non-null  int64  
 2   classe_bonus       310913 non-null  int64  
 3   idade              307270 non-null  float64
 4   is_casco           310913 non-null  float64
 5   is_rcdm            310913 non-null  float64
 6   is_rcdc            310913 non-null  float64
 7   is_dmoral          310913 non-null  float64
 8   uf                 310913 non-null  object 
 9   md_sexo            310913 non-null  object 
 10  RNS_Casco_calc     310913 non-null  int64  
 11  Grupo_veiculo_new  309319 non-null  object 
 12  idade_ult_RNS      310913 non-null  object 
 13  fabrica            310913 non-null  object 
 14  md_categoria       310913 non-null  object 
 15  MD_comb            310913 non-null  object 
 16  

In [189]:
# 
integer_columns = df_mod_glm_mercado.select_dtypes(include=['int64']).columns 
float_columns = df_mod_glm_mercado.select_dtypes(include=['float64']).columns 
object_columns = df_mod_glm_mercado.select_dtypes(include=['object']).columns 

In [190]:
float_columns=float_columns[1:]

In [191]:
print(integer_columns),print(float_columns),print(object_columns)

Index(['ano_mod', 'classe_bonus', 'RNS_Casco_calc'], dtype='object')
Index(['idade', 'is_casco', 'is_rcdm', 'is_rcdc', 'is_dmoral'], dtype='object')
Index(['uf', 'md_sexo', 'Grupo_veiculo_new', 'idade_ult_RNS', 'fabrica',
       'md_categoria', 'MD_comb', 'md_config', 'MD_Subzona', 'md_tipo_renov',
       'MD_franquia', 'md_menor26', 'MD_uso_veic'],
      dtype='object')


(None, None, None)

In [192]:
df_num = df_mod_glm_mercado[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
df_cat = df_mod_glm_mercado[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]

In [193]:
#df_cat_glm=pd.get_dummies(df_mod_glm_mercado[object_columns])
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_num, df_cat_glm],axis=1)

In [194]:
df_cat_glm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310913 entries, 355 to 571593
Columns: 607 entries, ano_mod to uf_TO
dtypes: float64(5), int64(3), uint8(599)
memory usage: 199.0 MB


In [88]:
#df_gbm_to_glm=pd.concat([df_mod_glm_mercado[integer_columns], df_mod_glm_mercado[float_columns],x_cat_glm],axis=1)

In [195]:
df_cat_glm.head(2)

,ano_mod,classe_bonus,idade,is_casco,is_dmoral,is_rcdc,is_rcdm,RNS_Casco_calc,Grupo_veiculo_new_ARRIZO,Grupo_veiculo_new_Audi,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
355,2005,10,70.0,24887.0,50000.0,400000.0,150000.0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
359,2003,4,36.0,22863.0,20000.0,100000.0,100000.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [72]:
#x_cat_glm.columns.to_list() y_pred = model_gbm.predict(x_test, predict_disable_shape_check=True)

In [196]:
GBM_mercado = model_gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [204]:
import numpy as np
np.average(GBM_mercado)

4064.208631369447

In [197]:
df_cat_glm['GBM_mercado'] = model_gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [198]:
display(df_cat_glm['GBM_mercado'].describe())

count    310913.000000
mean       4064.208631
std        1021.222322
min        2101.080266
25%        3315.287523
50%        3877.900563
75%        4618.329505
max       10571.766738
Name: GBM_mercado, dtype: float64

In [199]:
df_cat_glm.shape

(310913, 608)